In [1]:
import pandas as pd
import json
import requests
import warnings
warnings.filterwarnings("ignore")

쓰레기통 설치 현황 데이터 전처리

In [2]:
#현황데이터 불러오기
trash_map=pd.read_csv('데이터/trash_tool.csv',encoding='cp949') 

#설치장소 유형 컬럼 겹치는 내용 용어 통일
trash_map=trash_map.rename(columns={'설치 지점':'설치 장소 유형'}) 
trash_map=trash_map.replace({'설치 장소 유형':('도로(가로)변',' 도로(가로)변')},'도로변(횡단보도 포함)')
trash_map=trash_map.replace({'설치 장소 유형':(' 정류장(버스, 택시 등)', '정류소(버스, 택시 등)')},'정류장(버스, 택시 등)')
trash_map=trash_map.replace({'설치 장소 유형':(' 광장 등 다중집합장소', '광장, 공원 등 다중집합장소', '광장 등 다중집합장소')},'다중집합장소(광장, 공원 등)')

#결측치 확인
#trash_map.isnull().sum()

#필요한 컬럼만 추출
trash_map=trash_map.iloc[:,1:]

# 좌표에서 행정동 추출 코드
def geocoding_reverse(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    headers = {"Authorization": "KakaoAK cad14f63ba1708bba20c7ca6e334c490"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)
    행정동 = full_address['documents'][1]['region_3depth_name']
    return 행정동

for i in range(len(trash_map)):
    x = str(trash_map.loc[i,'x좌표'])
    y = str(trash_map.loc[i,'y좌표'])
    trash_map.loc[i,'행정동'] = geocoding_reverse(x, y)


trash_map[trash_map['행정동']=='안성1동']
#안성1동 오류 --> 혜화동으로 수정
trash_map.loc[169,'행정동']='혜화동'
trash_map.loc[170,'행정동']='혜화동'

#컬럼명 및 컬럼 순서 변경
trash_map=trash_map.rename(columns={'수거 쓰레기 종류(일반 쓰레기 / 재활용 쓰레기)':'수거 쓰레기 종류'})
trash=trash_map[['행정동', '세부 위치(상세 주소)', '설치 장소 유형','수거 쓰레기 종류']]

# 행정동별 쓰레기통 수
trash_sum=trash_map.groupby('행정동')['자치구명'].count().sort_values(ascending=False).to_frame().reset_index()
trash_sum=trash_sum.rename(columns={'자치구명':'쓰레기통 수'})
trash_sum=trash_sum.sort_values(by='행정동').reset_index(drop=True)
#trash_sum

trash

,행정동,세부 위치(상세 주소),설치 장소 유형,수거 쓰레기 종류
0,사직동,경복궁역 4번출구,지하철역 입구,일반쓰레기
1,사직동,경복궁역 4번출구,지하철역 입구,재활용
2,사직동,자하문로 28,도로변(횡단보도 포함),일반쓰레기
3,사직동,자하문로 28,도로변(횡단보도 포함),재활용
4,청운효자동,자하문로 44,도로변(횡단보도 포함),일반쓰레기
...,...,...,...,...
194,창신1동,종로300 동일빌딩앞,도로변(횡단보도 포함),일반쓰레기
195,창신2동,종로301 동일빌딩앞,도로변(횡단보도 포함),재활용
196,숭인2동,종로 413(우리은행앞),도로변(횡단보도 포함),일반쓰레기
197,숭인2동,종로 413(우리은행앞),도로변(횡단보도 포함),재활용


In [3]:
#중복제외 199개 입지

In [4]:
trash_sum

,행정동,쓰레기통 수
0,가회동,13
1,교남동,2
2,사직동,42
3,삼청동,15
4,숭인1동,1
5,숭인2동,2
6,이화동,10
7,종로1.2.3.4가동,78
8,종로5.6가동,6
9,창신1동,5


생활인구 데이터 전처리

In [5]:
# 생활인구 데이터
import pandas as pd

data_population=pd.read_csv('데이터/상권분석서비스 생활인구 데이터.csv',encoding='cp949') 
data_codes_information=pd.read_csv('데이터/종로구 상권코드.csv',encoding='cp949') #종로구 상권코드, 행정동 코드 데이터
seoul_codes=pd.read_csv('데이터/서울시 건축물대장 법정동 코드정보.csv',encoding='cp949') #서울시 행정동 코드, 시군구명, 행정동명에 관한 정보 데이터

#필요한 컬럼만 추출(위치에 따른 생활인구수에 관한 정보)
data_population=data_population.iloc[:,:7]

# 상권코드에 따라 종로구 데이터만 추출(생활인구 데이터)
data_population=data_population[(data_population['상권_코드']==1001494)|
           (data_population['상권_코드']==2110531)|
           ((data_population['상권_코드']>=2110001)&(data_population['상권_코드']<=2110037))|
            ((data_population['상권_코드']>=2120001)&(data_population['상권_코드']<=2120017))|
            ((data_population['상권_코드']>=2130001)&(data_population['상권_코드']<=2130021))
           ]

#상권코드를 기준으로 생활인구 데이터와 상권코드 데이터를 매핑하여 행정동 코드와 엑스좌표, 와이좌표값 알아냄
merge_pop= pd.merge(left=data_population, right=data_codes_information, how='inner', on='상권_코드')
merge_pop=merge_pop[['기준 년코드', '기준_분기_코드','상권_코드',' 상권_구분_코드_명', '상권_구분_코드_x', '상권_코드_명_x',
       '총_생활인구_수', '엑스좌표_값', '와이좌표_값', '시군구_코드', '행정동_코드']]



#2021년 데이터만 추출
merge_pop_2021=merge_pop[merge_pop['기준 년코드']==2021]
merge_pop_2021[merge_pop_2021['기준_분기_코드']==1].sort_values('상권_코드') #분기별 77개씩 

#상권코드 데이터에서 시각화에 필요한 행정동코드, 행정동명 추출
jongno_codes=seoul_codes[seoul_codes['시군구명']=='종로구']
jongno_codes=jongno_codes[['시군구코드','행정동코드','시군구명','행정동명']].drop_duplicates()
jongno_codes['행정동']=jongno_codes['시군구명']+' '+jongno_codes['행정동명']
jongno_codes=jongno_codes.astype({'시군구코드':'str'}) #데이터 타입 변경 str으로
jongno_codes=jongno_codes.astype({'행정동코드':'str'})
jongno_codes['행정동_코드']=jongno_codes['시군구코드']+jongno_codes['행정동코드']
jongno_codes=jongno_codes.astype({'행정동_코드':'int'}) #데이터 타입 다시 int로 변경
jongno_codes=jongno_codes.dropna(axis=0)
jongno_codes=jongno_codes[['행정동_코드','행정동']]
#jongno_codes.head()

merge_pop_2021= pd.merge(left=merge_pop_2021, right=jongno_codes, how='inner', on='행정동_코드')
pop=merge_pop_2021[['기준_분기_코드','상권_코드', '행정동','총_생활인구_수']]
#pop.head()

#행정동에 따른 총 생활인구수 그룹바이
pop_sum=pop.groupby('행정동')['총_생활인구_수'].sum().sort_values(ascending=False).to_frame().reset_index()

#행정동명만 추출
for i in range(len(pop_sum)): pop_sum['행정동'][i]=pop_sum['행정동'][i][4:]

#컬럼명 변경
pop_sum=pop_sum.rename(columns={'총_생활인구_수':'총 생활인구 수'})


In [6]:
pop_sum.head()

,행정동,총 생활인구 수
0,종로1.2.3.4가동,27171561
1,이화동,19447056
2,부암동,18044324
3,가회동,14668149
4,창신제1동,12862192


매출 데이터 전처리

In [7]:
#3. 매출데이터
data_sales=pd.read_csv('데이터/상권분석서비스 매출정보 데이터.csv',encoding='cp949')
data_sales=data_sales.iloc[:,:10]


# 상권코드에 따라 종로구 데이터만 추출(매출 데이터)
data_sales=data_sales[(data_sales['상권_코드']==1001494)|
           (data_sales['상권_코드']==2110531)|
           ((data_sales['상권_코드']>=2110001)&(data_sales['상권_코드']<=2110037))|
            ((data_sales['상권_코드']>=2120001)&(data_sales['상권_코드']<=2120017))|
            ((data_sales['상권_코드']>=2130001)&(data_sales['상권_코드']<=2130021))
           ]

merge_sales= pd.merge(left=data_sales, right=data_codes_information, how='inner', on='상권_코드')


merge_sales=merge_sales[['기준_년_코드', '기준_분기_코드','상권_코드','상권_구분_코드_명_x', '상권_구분_코드_x', '상권_코드_명_x',
       '서비스_업종_코드', '서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수', '엑스좌표_값', '와이좌표_값', '시군구_코드', '행정동_코드']]


merge_sales= pd.merge(left=merge_sales, right=jongno_codes, how='inner', on='행정동_코드')
sales=merge_sales[['기준_분기_코드','상권_코드', '행정동','분기당_매출_건수']]
#sales.head()

#행정동에 따른 총 매출건수 그룹바이
sales_sum=sales.groupby('행정동')['분기당_매출_건수'].sum().sort_values(ascending=False).to_frame().reset_index()

#행정동명만 추출
for i in range(len(sales_sum)): sales_sum['행정동'][i]=sales_sum['행정동'][i][4:]

#컬럼명 변경
sales_sum=sales_sum.rename(columns={'분기당_매출_건수':'총 매출 건수'})

In [8]:
sales_sum

,행정동,총 매출 건수
0,종로1.2.3.4가동,47291024
1,이화동,29944772
2,창신제1동,24263535
3,종로5.6가동,13965285
4,가회동,11673960
5,사직동,8591112
6,혜화동,5270009
7,교남동,5153420
8,청운효자동,4001699
9,부암동,3825932


버스정류소 데이터 전처리

In [9]:
#버스 정류소
import pandas as pd

data_bus=pd.read_csv('데이터/서울시 버스정류소 위치정보.csv',encoding='cp949')

#종로구 버스정류장만 추출
data_bus=data_bus[data_bus['정류소번호']<2000] #앞자리 1이 종로구 
data_bus=data_bus.rename(columns={'Y좌표':'x좌표'}) #위도 경도에 맞춰 x,y 좌표 수정
data_bus=data_bus.rename(columns={'X좌표':'y좌표'})
data_bus=data_bus[['정류소번호', '정류소명', 'x좌표','y좌표', '정류소 타입']] #필요한 컬럼만 추출


# 좌표에서 행정동 추출 코드
import requests
import json

def geocoding_reverse(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    headers = {"Authorization": "KakaoAK cad14f63ba1708bba20c7ca6e334c490"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)
    행정동 = full_address['documents'][1]['region_3depth_name']
    return 행정동

for i in range(len(data_bus)):
    x = str(data_bus.loc[i,'x좌표'])
    y = str(data_bus.loc[i,'y좌표'])
    data_bus.loc[i,'행정동'] = geocoding_reverse(x, y)

bus=data_bus[['행정동','정류소명','정류소 타입']]
#bus

#행정동별 버스 정류장 수
bus_sum=bus['행정동'].value_counts().to_frame().reset_index()
bus_sum=bus_sum.rename(columns={'count':'버스정류소 수'})
bus_sum

,행정동,버스정류소 수
0,종로1.2.3.4가동,101
1,평창동,59
2,사직동,40
3,청운효자동,38
4,부암동,36
5,혜화동,34
6,종로5.6가동,25
7,교남동,24
8,이화동,24
9,가회동,23


지하철역 데이터 전처리

In [10]:

#지하철역 데이터 불러오기
data_subway_address=pd.read_csv('데이터/서울교통공사 지하철역 주소 및 전화번호 정보.csv',encoding='cp949')
data_subway=pd.read_csv('데이터/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv',encoding='cp949')

#종로구에 위치한 지하철역만 추출
jongno_subway_station=list(data_subway_address[data_subway_address['도로명주소'].str.contains('종로구')]['역명'])
#jongno_subway_station

#종로에 있는 역만 data_subway에서 추출하고 주소랑 조인하기
subway_map_jongno=data_subway[data_subway['역명'].isin(jongno_subway_station)]
subway_map_jongno=subway_map_jongno.reset_index()
subway_map_jongno.head()

#종로구 지역 지하철역의 주소만 추출
subway_address_jongno=data_subway_address[data_subway_address['도로명주소'].str.contains('종로구')]

#호선 글자 없이 숫자만 남기기(조인하기위해)
for i in range(len(subway_address_jongno)):
    subway_address_jongno.iloc[i]['호선']=subway_address_jongno.iloc[i]['호선'][0]

    
#두 데이터셋에서 같은 역명 통일
subway_map_jongno.loc[10,'역명']='종로3가(탑골공원)'

#조인하기 위해 데이터 타입 통일
subway_address_jongno=subway_address_jongno.astype({'호선':'int'})
#subway_address_jongno.info()

#호선, 역명을 기준으로 지하철역 위도경도 데이터와 주소 데이터 조인
data_subway= pd.merge(left=subway_address_jongno, right=subway_map_jongno, how='left', on=['호선','역명'])
data_subway=data_subway[['호선','역명','위도','경도','도로명주소']]

#3호선 경복궁역, 5호선 광화문역 위도경도 정보 입력(구글지도 참고)
data_subway.loc[5,'위도']=37.575838
data_subway.loc[5,'경도']=126.97358
data_subway.loc[11,'위도']=37.571665
data_subway.loc[11,'경도']=126.976502


# 좌표에서 행정동 추출 코드
def geocoding_reverse(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    headers = {"Authorization": "KakaoAK cad14f63ba1708bba20c7ca6e334c490"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)
    행정동 = full_address['documents'][1]['region_3depth_name']
    return 행정동

for i in range(len(data_subway)):
    x = str(data_subway.loc[i,'위도'])
    y = str(data_subway.loc[i,'경도'])
    data_subway.loc[i,'행정동'] = geocoding_reverse(x, y)

subway=data_subway[['행정동','호선','역명','도로명주소']]
#subway

#행정동별 지하철역 수
subway_sum=subway['행정동'].value_counts().to_frame().reset_index()
subway_sum=subway_sum.rename(columns={'count':'지하철역 수'})
subway_sum

,행정동,지하철역 수
0,종로1.2.3.4가동,4
1,창신1동,3
2,사직동,2
3,종로5.6가동,1
4,숭인1동,1
5,삼청동,1
6,혜화동,1
7,충현동,1
8,창신3동,1


데이터 합치기

In [11]:
# 종로구 총 17개 행정동에 따른 데이터 합치기
# 생활인구, 매출, 버스정류소, 지하철역 데이터 합치기
# 쓰레기통 현황 데이터와 비교

#행정동 기준으로 데이터셋 합치기
pop_and_sales=pd.merge(pop_sum,sales_sum,on='행정동',how='outer')
bus_and_subway=pd.merge(bus_sum,subway_sum,on='행정동',how='outer')
dataset=pd.merge(pop_and_sales,bus_and_subway,on='행정동',how='outer')
dataset=dataset.drop([22], axis=0) # 충현동--> 서대문구 지역 삭제

#같은 행정동끼리 데이터 통일
dataset.loc[19,'총 생활인구 수']=dataset.loc[13,'총 생활인구 수'] #숭인1동
dataset.loc[19,'총 매출 건수']=dataset.loc[13, '총 매출 건수']
dataset.loc[18,'총 생활인구 수']=dataset.loc[7,'총 생활인구 수'] #숭인2동
dataset.loc[18,'총 매출 건수']=dataset.loc[7, '총 매출 건수']

dataset.loc[17,'총 생활인구 수']=dataset.loc[4,'총 생활인구 수'] #창신1동
dataset.loc[17,'총 매출 건수']=dataset.loc[4, '총 매출 건수']

dataset.loc[21,'총 생활인구 수']=dataset.loc[10,'총 생활인구 수'] #창신2동
dataset.loc[21,'총 매출 건수']=dataset.loc[10, '총 매출 건수']

dataset.loc[20,'총 생활인구 수']=dataset.loc[11,'총 생활인구 수'] #창신3동
dataset.loc[20,'총 매출 건수']=dataset.loc[11, '총 매출 건수']

#겹치는 행 삭제
dataset=dataset.drop([4,7,10,11,13], axis=0) # 창신제1동,창신제2동,창신제3동, 숭신제1동,숭신제2동

#nan 값 0으로 채우기 --> 지하철역 수가 0개인 경우이므로
dataset=dataset.fillna(0)
dataset=dataset.sort_values(by='행정동').reset_index(drop=True)
dataset



,행정동,총 생활인구 수,총 매출 건수,버스정류소 수,지하철역 수
0,가회동,14668149.0,11673960.0,23.0,0.0
1,교남동,1157553.0,5153420.0,24.0,0.0
2,무악동,737944.0,511342.0,4.0,0.0
3,부암동,18044324.0,3825932.0,36.0,0.0
4,사직동,10147192.0,8591112.0,40.0,2.0
5,삼청동,708366.0,2081240.0,16.0,1.0
6,숭인1동,1471257.0,167153.0,13.0,1.0
7,숭인2동,7907348.0,2546202.0,13.0,0.0
8,이화동,19447056.0,29944772.0,24.0,0.0
9,종로1.2.3.4가동,27171561.0,47291024.0,101.0,4.0


In [12]:
# csv 파일로 저장

#현황데이터/현황sum
# trash_map.to_csv('데이터/trash_map.csv',index=False,encoding='cp949')
# #trash.to_csv('데이터/trash.csv',index=False, encoding='cp949')
# #trash_sum.to_csv('데이터/trash_sum.csv',index=False, encoding='cp949')

# #생활인구데이터/생활인구sum
# pop.to_csv('데이터/pop.csv',index=False, encoding='cp949')
# pop_sum.to_csv('데이터/pop_sum.csv',index=False, encoding='cp949')
#merge_pop_2021.to_csv('데이터/merge_pop_2021.csv',index=False,encoding='cp949')

#매출데이터/매출sum
# sales.to_csv('데이터/sales.csv',index=False, encoding='cp949')
# sales_sum.to_csv('데이터/sales_sum.csv',index=False, encoding='cp949')
#merge_sales.to_csv('데이터/merge_sales.csv',index=False,encoding='cp949')

#버스/버스sum
#data_bus.to_csv('데이터/data_bus.csv',index=False,encoding='cp949')
#bus.to_csv('데이터/bus.csv',index=False, encoding='cp949')
#bus_sum.to_csv('데이터/bus_sum.csv',index=False, encoding='cp949')

#지하철역/ 지하철역sum
#data_subway.to_csv('데이터/data_subway.csv',index=False, encoding='cp949')
#subway.to_csv('데이터/subway.csv',index=False, encoding='cp949')
# subway_sum.to_csv('데이터/subway_sum.csv',index=False, encoding='cp949')

#취합데이터(생활인구, 매출, 버스, 지하철)
#dataset.to_csv('데이터/dataset.csv',index=False, encoding='cp949')
